In [1]:
!pip install selenium

     ---------------------------------------- 9.4/9.4 MB 12.3 MB/s eta 0:00:00
     -------------------------------------- 475.7/475.7 kB 9.9 MB/s eta 0:00:00
     ---------------------------------------- 58.8/58.8 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 0.58.0
    Uninstalling websocket-client-0.58.0:
      Successfully uninstalled websocket-client-0.58.0
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.2.0
    Uninstalling sniffio-1.2.0:
      Successfully uninstalled sniffio-1.2.0
  Attempting uninstall: attrs
    Found existing insta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.


<iframe id="searchIframe" title="title_name" src="blahlala"></iframe>

In [15]:
from audioop import add
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import string
import time
import pandas as pd
import re

url_list = []
data_id = []
datas = []


def infinite_loop(driver):
    # 최초 페이지 스크롤 설정
    # 스크롤 시키지 않았을 때의 전체 높이
    last_page_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 윈도우 창을 0에서 위에서 설정한 전체 높이로 이동
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.0)
        # 스크롤 다운한 만큼의 높이를 신규 높이로 설정 
        new_page_height = driver.execute_script("return document.body.scrollHeight")
        # 직전 페이지 높이와 신규 페이지 높이 비교
        if new_page_height == last_page_height:
            time.sleep(1.0)
            # 신규 페이지 높이가 이전과 동일하면, while문 break
            if new_page_height == driver.execute_script("return document.body.scrollHeight"):
                break
        else:
            last_page_height = new_page_height
            
            
def naverMapCrawling(search):

    #드라이버 설정
    options = Options()
    #오히려 헤드리스 설정 쓰니까 못 긁어오는 경우가 있더라구요.
    #options.add_argument("--headless")
    #options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options = options)

    # 웹 네이버 지도 https://map.naver.com/p/search/%EC%9D%B4%EB%8C%80?c=15.00,0,0,0,dh
    driver.get(f"https://map.naver.com/p/search/{search}") 
    driver.implicitly_wait(10) # 로딩이 끝날 때 까지 10초까지는 기다림
    
    #iframe 내 요소 가져오기! 
    iframe = driver.find_element(By.CSS_SELECTOR, 'iframe#searchIframe')  # iframe의 적절한 선택자로 변경
    driver.switch_to.frame(iframe)
    #페이지마다 눌러서 아이디 뽑아오기!
    buttons = driver.find_elements(By.CLASS_NAME, 'mBN2s')
    #몇 페이지 있는지 확인 
    print(len(buttons))
    
    for i in range(len(buttons)):
        try:
            # 특정 클래스를 가진 모든 요소 찾기
            print(buttons[i])
            if buttons[i]:
                buttons[i].click()
                time.sleep(3)  # 클릭 후 잠시 대기 (필요에 따라 조절)
                driver.switch_to.default_content() #다시 원래 프레임으로!
                #sc-q8es3i가 클래스 이름에 들어가는 요소들을 뽑아오기(id)
                #클래스 이름이 조금씩 다르고 sc-q8es3i부분만 공통적으로 되어있음!
                #지도 부분에서 id 뽑아오기!! 
                items = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.sc-q8es3i'))
                    )
                print(len(items))
                print("##################NEW PAGE##################")
                
                #data-title과 같은 어트리뷰트가 모바일에만 있는건지... 안보여서 주석
                #하려고 했지만 어찌저찌 찾았습니다! 와아!!!!
                for item in items:
                    id_get = item.get_attribute('id')
                

                    # 정규식을 사용하여 id 속성에서 숫자만 추출
                    match = re.search(r'\d+', id_get)
                    if match:
                        id = match.group()
                        id = int(id)

                    datas.append({
                        "id" : id
                        })

                    data_id.append(id)
                if i<len(buttons):
                    iframe = driver.find_element(By.CSS_SELECTOR, 'iframe#searchIframe')  # iframe의 적절한 선택자로 변경
                    driver.switch_to.frame(iframe)

            #else:
            #    print("Not enough buttons to click.")
            #    break

        except Exception as e:
            print(f"An error occurred during clicking button {i + 1}: {e}")
            break

    
    for id in data_id:
        # 홈 탭으로 이동
        driver.get(f'https://m.place.naver.com/restaurant/{id}/home')
        time.sleep(1)
        
        
        # 가게 이름 
        try: 
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.GHAhO')))
            title = driver.find_element(By.CLASS_NAME, 'GHAhO').text   #CLASS_NAME으로 찾기
            print(title)
    
        except:
            title = ""
            print("없는 ID")

        for data in datas:
            if data["id"] == id:
                data["title"] = title
                break
                
                
        # 별점
        try: 
            score = driver.find_element(By.CSS_SELECTOR, '.LXIwF').text
            score_num = float(re.findall("\d+.\d+",score)[0]) # 데이터에서 별점만 추출
            print(score_num)
        except:
            score_num = ""
            print("별점 없음")

        for data in datas:
            if data["id"] == id:
                data["score_val"] = score_num
                break
     
        # 음식 카테고리
        try: 
            category = driver.find_element(By.CSS_SELECTOR, 'span.lnJFt').text
            print(category)
        except:
            category = ""
            print("카테고리 없음")

        for data in datas:
            if data["id"] == id:
                data["category"] = category
                break
        
        # 가게 설명글
        try: 
            descript = driver.find_element(By.CSS_SELECTOR, 'div.XtBbS').text.strip("'\"")
            descript = descript.replace("\n", " ") # 중간에 있는 엔터 없애기
            print(descript)
        except:
            descript = ""
            print("가게설명 없음")

        for data in datas:
            if data["id"] == id:
                data["descript"] = descript
                break
        
        # 리뷰 탭으로 이동
        driver.get(f'https://m.place.naver.com/restaurant/{id}/review/visitor')
        time.sleep(1)
        # 대표 키워드 (3개)
        try: 
            keyword1 = driver.find_elements(By.CSS_SELECTOR, 'span.t3JSf')[0].text.strip("'\"")
            keyword2 = driver.find_elements(By.CSS_SELECTOR, 'span.t3JSf')[1].text.strip("'\"")
            keyword3 = driver.find_elements(By.CSS_SELECTOR, 'span.t3JSf')[2].text.strip("'\"")

            print(keyword1)
            print(keyword2)
            print(keyword3)
        except:
            keyword1 = ""
            keyword2 = ""
            keyword3 = ""
            print("키워드 없음")

        for data in datas:
            if data["id"] == id:
                data["keyword1"] = keyword1
                data["keyword2"] = keyword2
                data["keyword3"] = keyword3
                break
        
        # 긴 리뷰 펼치기
        try:
            extension_btn = driver.find_elements(By.CSS_SELECTOR, 'span.rvCSr')
            for btn in extension_btn:
                btn.click()
        except:
            print("긴 리뷰 없음")

        # 리뷰(10개)
        try:             
            reviews = ""
            for i in range(10):
                try:
                    review = driver.find_elements(By.CSS_SELECTOR, 'span.zPfVt')[i].text
                    if(i == 0):
                        reviews = f'{review}' # 리뷰를 reviews에 누적
                    else:
                        reviews = f'{reviews} / {review}' # 각 리뷰는 /로 구분
                except IndexError:
                    print("리뷰가 충분하지 않음") # 리뷰가 5개 이하인 가게
                    break
            reviews = reviews.replace("\n", " ") # 리뷰 중간 엔터 제외
            print(reviews)
        except Exception as e:
            reviews = ""
            print("리뷰 없음")

        for data in datas:
            if data["id"] == id:
                data["reviews"] = reviews
                
                break
    
    # 결과 데이터프레임 리턴하기 
    df = pd.DataFrame(datas, columns=["id", "title", "score_val", "category", "descript", "reviews", "keyword1", "keyword2", "keyword3"])
    return df


In [6]:
# 함수 실행
search = "신촌 맛집" # 여기에 검색어 입력
df1 = naverMapCrawling(search)

print("done")

5
<selenium.webdriver.remote.webelement.WebElement (session="de083dd5c4c2fe7dad84073b61b63106", element="f.36D794B41ECFE8E6140CD5ED445D0E72.d.347B4679D7AD273636D57F4391900E09.e.127")>
72
##################NEW PAGE##################
<selenium.webdriver.remote.webelement.WebElement (session="de083dd5c4c2fe7dad84073b61b63106", element="f.36D794B41ECFE8E6140CD5ED445D0E72.d.347B4679D7AD273636D57F4391900E09.e.128")>
73
##################NEW PAGE##################
<selenium.webdriver.remote.webelement.WebElement (session="de083dd5c4c2fe7dad84073b61b63106", element="f.36D794B41ECFE8E6140CD5ED445D0E72.d.347B4679D7AD273636D57F4391900E09.e.129")>
68
##################NEW PAGE##################
<selenium.webdriver.remote.webelement.WebElement (session="de083dd5c4c2fe7dad84073b61b63106", element="f.36D794B41ECFE8E6140CD5ED445D0E72.d.347B4679D7AD273636D57F4391900E09.e.130")>
70
##################NEW PAGE##################
<selenium.webdriver.remote.webelement.WebElement (session="de083dd5c4c2fe7dad8

현명식탁
4.72
양식
농림축산식품부 제공 안심식당
음식이 맛있어요
친절해요
매장이 넓어요
리뷰가 충분하지 않음
부모님 모시고 감 샐러드, 크림파스타, 피자 다 너무 잘드심 친절하고 스테이크도 맛있고 추천 / #파스타맛집 #신촌맛집 #신촌파스타 직원분들 정말 친절하고 파스타가 맛있어서 추천!! 솔직히 스테이크는 그냥 그랬구 까르보나라가 쫀맛탱!! 직원분들 넘 친절해서 괜히 기분 좋았어요! 번창하세요! :-) / 청첩장모임하러 왔는데 분위기도 좋고 친절하시고 음식도 다 너무 맛있어서 기분 좋은 식사하고 가요~~ 스테이크도 먹기 좋게 썰어주시고 메뉴 설명도 해주셔서 세심한 서비스가 너무 좋았어요~~ 다음에 또 방문하겠습니다! / 친구들이랑 여행와서 예약하고 들렸어요. 예약하면 샤베트를 따로 주시더라고요! 오픈하자말자 바로 와서 그런지 한적하게 잘먹었습니다. 직원 분들도 친절하시고 전체적으로 다 맛있었는데 그중 감자튀김이 정말 맛있었요...^_^ 다음에 또 들릴게요 / 콜키지프리로 와인가져가서 먹었어용~ 살치살 스테이크는 생각보다 힘줄이 많이 씹혀서 질겼어요 ㅠㅠ 하지만 원하는 대로 굽기를 구워서 먹을 수 있어서 좋네요 🧡 트러플 크림 파스타 짱 맛있네요 👍🏻👍🏻 친구랑 저랑 완전 파스타 맘에 들었어요 ❤️❤️ / 기대했던 대로 분위기 좋고 음식이 맛있어서 좋은 시간을 보내다갑니다. 😊 다이닝 레스토랑에서 스테이크를 직접 구워서 먹으라는 얘기를 들었을 때는 다소 당황했는데, 금방 오셔서 직접 구워주셔서 다행이다 싶었습니다. 다른 손님하고 차별대우하나 싶었네요. / 친구 생일겸 방문했었는데 음식도 넘 맛있구 우선 스테이크가 정말 넘 맛있었습니다 👍🏻직원분들께서도 넘 친절하시고 친구생일 같이 축하해주셔서 넘 감사했어요ㅠㅠ🤍 리뷰 꼭 쓰고싶었는데 이제 올립니당 ㅎㅎ 매장이 정말 넓고 테이블도 단체석까지 완비되있으니 모두 현명식탁 방문해보세요! 강추합니당 !! / 셋이서 배가 많이 고프지 않아 메뉴 두개를 시켰는데 파스타 양을 두배로 주시고 감자튀김도 서비스로 주셨어요!!

10593 베이글커피하우스 신촌점
별점 없음
카페,디저트
가게설명 없음
디저트가 맛있어요
인테리어가 멋져요
커피가 맛있어요
리뷰가 충분하지 않음
매장이 넓으면서 소품들이 아기자기하게 잘 꾸며져있고 사진도 잘 나오고 예뻐요~ 단밤라떼 달다고 하는 분들도 있는데 전 단것도, 밤을 너무 좋아해서 맛있더라구요🌰🤎 커피도 적당히 밸런스도 좋아요 치즈믹스베이글은 빵도 쫀득하고 괜찮았는데 생각보다 치즈 맛이 많이 안나서 아쉬웠어요 / 신촌에 이런 곳이?! 2층으로 되어 있어 넓고 자리도 많아, 대화하러 오거나 작업하기도 좋은 공간이다 인테리어도 컨셉에 맞게 예쁘게 되어있고, 음식음료 다 맛있다. 특히 애플망고빙수는 아래 씨리얼까지 있어서 더 풍미가 느껴진다 화장실도 깨끗 / 베이글이 너무 먹고싶어서 찾아보다 방문했는데 이게 머선일이죠 너무 맛있어요 고구마 치즈 베이글 강츄 ❤️완전 제스타일 콘베이컨 샌디치도 담백하구 괜춘 하지만 원픽은 고치베 그리고 토마토 스프도 꼭 드세욘 매콤해서 느끼함 싹 잡아줌 쵝오 .. 서울오면 또 올게요오오 잘먹구 갑니당 🐣 / 이름은 베이글하우스지만 제발 수프를 드셔주세요ㅠㅠㅠㅠㅠㅠ 토마토 수프 그냥 토마토 그 자체인데 매콤해서 너무 감칠맛나고 맛있어요 그리고 크림치즈 진짜 안달고 너무 꾸덕하고 미쳤습니다 진짜 그동안 달아빠진 크림치즈를 먹은 제가 너무 불쌍해요 매장도 넓고 자리도 편하고 쾌적해요 근데 진짜 토마토 수프를 드셔주세요 ****토마토 수프**** 괜히 평일쿠폰으로 수프를 주시는게 아니에요 ********토마토수프******** / 2번째 방문이에요!! 전에 먹구 너무 맛있어서 또왔어요>< 아직 다 도전해보진 않았지만 블루베리랑 황치즈가 정말 맛있는거같아요  스프도 생각치못했는데 조합도 좋구 꽤 맛있었어요 담엔 토마토스프 도전하러 또올게요! / 바질 베이글 + 바질 크림치즈 추천하고,, 통밀 베이글 with 토마토숲 진짜 꼭 드세요 ... 너무너무너무너무너무너무 맛있습니다ㅜㅜ 너무 달고 짠 빵에 점령당한 코리아.. 너무 맛있어요 ! 

풍년닭도리탕 신촌직영점
별점 없음
닭볶음탕
가게설명 없음
음식이 맛있어요
친절해요
양이 많아요
리뷰가 충분하지 않음
오버 안하고 살면서 먹은 닭도리탕중 가장 맛있었던 것 같아요..ㅋㅋㅋㅋ 풍년닭도리탕 원래 좋아해서 본점, 다른 지점 많이 가봤거든요! 근데 본점보다도 더 맛있어욬ㅋㅋㅋㅋㅋㅋ어떻게 이러지,,,  하 고기가 퍽퍽한 부위가 하나도 없고ㅜㅜ 육수 예술입니다,,, 최고최고 / 우와~우선 양이 엄청 많아서 깜짝 놀랐어용~양도 많은데 또 엄청 맛있었어요 🥰 국물이 진짜 맛있고 저 퍽퍽살 안먹는데 퍽살도 부드럽더라구요❤🧡원래 날개만 먹는데 걍 다 먹어버렸어요😁넘나 맛있어서 대낮부터 막걸리도 시켜버림💕쫄면사리에 밥까지 볶아먹고 왔어요~같이간 친구도 맛있다고 엄청 먹었어요👍👍사장님이 엄청 친절하셔서 기분좋게 더 맛있게 먹고 왔어용💯💯💯담에 또 방문할게용 😁 / 신촌 닭볶음탕 맛집 풍년닭도리탕! 4인이서 방문해서 닭도리탕 대자에 공기밥4개, 쫄면사리 추가해서 먹었는데 너무너무 맛나요! 매콤얼큰 감칠맛 가득한 양념에 잡내 없이 잘 삶아진 닭, 푹 익은 감자까지! 게다가 요 쫄면사리는 꼭 추가해서 먹어야 해요! 닭볶음탕에 쫄면이 이렇게 잘 어울리는지 몰랐어요🤍 음식도 빠르게 나오고 호불호 없이 맛있는 맛이라 만족했어요! 또 방문할게요🤍 / 인생 닭도리탕 찐맛집을 찾았습니다!! 일단 양이 완전 혜자구요 닭도리탕 소짜리 시켰는데 3인도 충분히 먹을 수 있는 양이예요! 아까아까 먹고왔는데 지금도 너무 배가 불러요😋 그리고 쫄면사리는 100% 무조건 시켜야합니다 라면사리와는 또 다른 매력이고 사리도 양 어마어마하죠? 간이 자극적이지 않은데 매콤해서 질리지 않고 배불러도 계속 들어가요 ㅠㅠ 닭고기도 정말 부드럽고 퍽퍽살 마저도 부드러워서 입에서 살살 녹아요☺️☺️ 사장님도 엄청 친절하시구 매장도 깔끔하고 에어컨 완전 빵빵👍🏻 전 추웠어요 ㅋㅋㅋ 수요미식회에도 출연한 신동엽 맛집이라고해요😃 더위 피해서 매콤한 닭도리탕 꼭 드셔보세요!! 너무 맛있어서 다음에 또 방문할거예요 인생 

치히로 신촌점
별점 없음
일식당
가게설명 없음
음식이 맛있어요
인테리어가 멋져요
친절해요
리뷰가 충분하지 않음
처음 방문해봤네요. 저는 양념된 대창이 올라간 호르몬동을 주문했고, 일행은 텐동으로 주문했습니다. 같이 먹을 아부리반반초밥도 시켰고요. 호르몬동은 겉은 바삭하고 안에는 기름기 가득한 식감이었고, 계란을 올려서 먹으니 더 녹진하고 좋았네요. 텐동은 새우튀김만 받아서 먹어봤는데 맛 좋았습니다. / 텐동 호르몬동 둘다 양도 많고 맛있었어요!! 호르몬동은 살짝 매콤한 양념이 있어요. 떡꼬치 정도의 매운맛?? 노른자랑 대창 비벼먹으면 맛있어요. 텐동은 같이 준 와사비마요에 찍어먹으니 맛있었습니다. 다음에 근처 지날 일 있으면 또 생각날 것 같아요 ! / 너무너무 맛있습니닷 ㅋ 처음 먹었을 때 너무 맛있게 먹어서 혼밥하러도 오고 오늘은 또 다른 지인 델꼬 또 먹었어요 ㅋ 처음에는 먹부심에 양이 적어보였는데 먹다보니 배가 너무 부르고 음식을 남길정도로 양이 아주 넉넉합니다. 직원분들도 친절하고 너무 좋아요~ 맛있게 잘 먹었습니다~ / 치히로 전부터 가보고 싶었는데 새로오픈한 신촌점으로 다녀왔어요! 입구부터 매력적인 치히로 신촌점🫰 음식도 다양하고 양도 많고 맛도 좋아요~! 가게 내부도 분위기가 좋아서 맛있는 음식과 함께 시간보내기 좋았어요❣️ 호르몬동이랑 텐동 강추강추🫶 / 맛집 불모지인 신촌에 괜찮은 식당이 보여서 다녀왔어요! 이 근방에 혼밥할 곳이 없어서 안타까웠는데 2인 좌석이 있어서 부담없이 먹을 수 있었습니다:) 단체로나 데이트하러도 많이 오시더라고요! 연어 뱃살 시켰는데 기름져서 맥주가 땡기는 맛이었습니다 ㅎㅎ 완전 강추드려요👍 / 남자친구랑 오랜만에 데이트하러 멀리 나왔는데 뭐 먹을지 고민하다가 평소 잘 먹지 않는 텐동이 생각나서 찾아보다 방문한 곳입니다.  사실 저는 텐동을 먹가보면 중간에 느끼해서 잘 안먹었었는대 치히로 텐동은 바삭하고 밥 간이 잘 되어있어서 끝까지 맛있게 먹었습니다! 호르몬동도 매콤달달하니 느끼하지 않고 소스도 넉넉해서 끝까지 먹

...

스타벅스 신촌점
4.48
카페
식약처 인증 위생등급 매우우수 음식점
커피가 맛있어요
음료가 맛있어요
친절해요
리뷰가 충분하지 않음
그동안 딸기 아사이레모네이드 많이 마셨는데 카페인 성분이 높다는걸 오늘 첨으로 알았어요. 알려주신 직원분 이름이라도 알아올걸 그랬어요. 정말 친절하게 알려주시면서,카페인 많은데 괜찮으시냐고 까지 물어봐 주셨거든요~ 감사합니다! / 신촌세브란스 병원 들렸다가 시간이 남아서 커피 한잔 마시려고 잠시 들렸다 왔는데 학생들이 많아요~ 생크림 카스테라 맛있게 먹었네요~ 오늘은 음료가 빨리 나와서 기분 좋게 마셨어요~ / 식사 후 커피는 늘 만족스러워요. 오늘도 맛있어서 좋아요. / 세브란스 갓다가 돌아오면서 항상 들리는 신촌 스벅 조아용 🥰👍🏻 / 명불허전 치즈케익 찐맛 / 희안하게 누가 놀러오면 스벅 쏘게되넨(?) ㄷㄷㄷ / 매장이 3층까지있어서 죄석이 많네요. 화장실도 깨끗해요. / 딸기라떼가 맛있대요 밀크티도 좋아요 / 쿠폰 있어서 사먹었어요~^^ / 신촌로터리 벚꽃길 테이크아웃해서먹었어요
해누리
별점 없음
생선회
가게설명 없음
음식이 맛있어요
재료가 신선해요
친절해요
리뷰가 충분하지 않음
숙성회가 맛있다고 해서 평일 저녁에 예약하고 방문했어요. 숙성회 2인세트 주문했는데 한상 가득입니다ㅎ 숙성회, 참치, 해산물세트까지 푸짐하고 예쁘게 나와요. 개인적으로 활어회보다는 숙성회를 좋아하는데 식감이 쫄깃하고 비린 맛이 전혀 없었어요.회로 배채우기 쉽지 않은데 회가 두툼하고 커서 둘이서 배부르기 먹었어요. 해산물들은 엄청 신선했구요.  반찬들도 맛있었어요. 때에 따라 다르게 나온다는데 관전전이 별미였어요~ 그리고 어제 꽤 더웠는데 시원달달한 묵사발도 별미였어요.  사장님도 친절하시고 숙성회를 좋은 가격에 맛볼 수 있어서 만족스러운 식사였어요!! / 차분한 분위기와 조용히 식사하기 좋았던 해누리! 👍🏻숙성회 2인세트 먹었어요 🍣 구성도 알차고 신선하고 맛있었어요 :) 숙성회 + 참치 + 해산물 다양하게 맛 볼 수 있어요 🙃 회들도 엄청 두툼하게

후쿠란 상수본점
별점 없음
이자카야
가게설명 없음
음식이 맛있어요
친절해요
재료가 신선해요
리뷰가 충분하지 않음
요즘 가장 자주 방문하는 식당-후쿠란  매번 주문하는 안키모와 우니크림파스타  입맛 까다로운 장모님도 크림파스타 밸런스가 딱 적당하다고 하시네요 ㅎㅎㅎ  쿠폰 불초밥까지 완벽한 식사하고 갑니다!  또 올게요 / 친구들과 오랜만에 놀러온 합정!! 후쿠란에 왔어요ㅋ 역시나 소문대로 분위기 너무좋고~ 모듬사시미 진짜맛있어요ㅋ 2차로온건데 바지락술찜도 진짜맛있고 하이볼도 너무맛있어요^^ 다음에도 꼭방문할거에요~ / 간단하게 친구랑 먹으러 갔는데 다음에 또 가고 싶을 만큼 만족스러워요 ! 우선 2층에 테이블이 많아서 자리도 넓고, 분위기도 넘 좋더라고요 ! 초밥도 큰 편이어서 배부르게 먹을 수 있어 좋았어요 ㅎㅎ 닭목살 가라아게도 소스랑 같이 찍어먹으면 두 배로 맛있더라고요.. 두 메뉴 모두 추천해요 ! 상수역 분위기 맛집을 찾으신다면 추천드려요 !! / 1층 다찌석이고 2층 테이블석인데 제가 갔을때는 세테이블 정도 손님이 있으셨고 어느 새 2층도 다차나 싶더니 먹고 계산하러 1층 내려갔더니 다찌석까지 만석이었어요! 드셔보시면 그 이유를 알 수 있답니다! 숙성회가 친구는 자기가 먹은 데 중 최고라고 하고 저 역시 두툼한 회가 식감까지 좋았어요! 그리고 명란 계란말이 꼭 드세요! 진 ~ 짜 맛있답니다😀👍🏼 / 후쿠란 밤에만 와봤는데 점심 특선 초밥과 ,카이센동 특선 먹구 우동도 서비스로 먹어봤눈데 서비스 우동 조차 맛있네요 일본이 생각 절로 나용 여기 횟감은 항상 굳굳 몇달전에 지인 소개 시켜줬는데 지인도 와보고 맛있다고 또 오고 싶다구 해서 만족 스러워요 오래오래 계셔주세여! / 저희 부부가 함께 좋아하는 얼미부부 영상에서 보고 저장해뒀다가 갔어요 얼과 미가 왜 진실로 화가났는지 알 것 같아요 진짜 너무 너무 맛있습니다 진실의 미간을 보실 수 있어요 회가 소고기보다 맛있다니 숙성회인가봐요 정말 너무너무 달고 부드럽고 맛있습니다!! 무조건 가성비 세투 드세요 

미정국수0410 멸치국수잘하는집신촌점
별점 없음
국수
신촌 미정국수에서 먹어본 멸치국수
음식이 맛있어요
가성비가 좋아요
혼밥하기 좋아요
리뷰가 충분하지 않음
일년 만에 온 거 같은데 가격은 살짝 오른 것 같지만(아닌가!?) 퀄리티가 좋아져서 맛있게 먹었습니다❣️신촌점이 특히 맛있는 거 같아요! / 영상 찍는 것에 정신 팔려서 사진 남긴 게 없음..  땅콩초계국수 + 갈비쫀득해만두  의 조합은?  땅콩+겨자소스 묘한데 맛있어! 겨자소스 싫어하시거나 코 매워하시는 분은 미리 말씀해주시면 덜어주시거나 아예 빼주십니다! 꼭꼭 요청을   갈비쫀득해 만두는 정말 쫀득하고  크기가 미니미니해서 같이 먹기 좋아용 / 3번째 오고 있습니다! 가성비 ㄱㅊ고 맛나요 / 처음 방문했는데 가격, 맛 모두 만족했습니다.  혼자 점심 식사 하기에 좋은듯 해요 / 가격대비 맛좋고 매장도 깨끗하네요 / 가성비 댜박~~집…맛나뇨..짱 / 초계비빔면 존맛입니다 3번째 가는 중 가성비 좋고 맛있어여 / 가성비가 좋아요 여기 멸치국수가 찐 맛임! 제육덮밥도 양 많고 맛있었네용 / 교환권 올려놨는데 선풍기 바람이 세서 날아간걸 왜 옮긴 자리에 안옮겨 놨냐고 계속  면박 주시는데 기분이 상당히 불쾌했네요  날아간 교환권이 제 탓인가요? 사방이 선풍기 바람인데..  나중에 교환권 찾고 교환권 이렇게 생긴거다 다시 보여주시는거 보고 참 어이 없었습니다 그게 그렇게 중요한일인가요? 살다보면 교환권 날아 갈 수도 있지 않습니까? / 양이 좀적어요 맛은좋아요
오시리야
별점 없음
일식튀김,꼬치
홍대 이자카야 맛 좋은 꼬치
음식이 맛있어요
친절해요
재료가 신선해요
리뷰가 충분하지 않음
매일 직접 만드는 수제 꼬치구이 20년 숙성된 소스와 노하우로 비장탄에 직접 구운 모듬꼬치✨  오시리야만의 특제소스를 발라 비장탄에 노릇노릇 구워진 비주얼🔥 보기만 해도 육즙 팡팡 느껴져요  간장을 비롯한 양념을 발라 달달하면서 짭짤함까지 느껴지는 단짠단짠 마성의 수제 소스🫶🏻  큼지막하면서 튼실한 장어구이에 후숙 잘 된 아보카도 

나이스샤워
4.48
일식당
서울텐동 먹고싶어 신촌 연대맛집
음식이 맛있어요
혼밥하기 좋아요
양이 많아요
리뷰가 충분하지 않음
나이스 샤워 텐동 맛있어요🤎🫶🏽🤤 튀김은 부드럽게 바삭바삭하고요, 튀김 재료들 전부 큼직큼직해서 양도 많아가지구 완-전 배부르게 드실 수 있어요! 텐동이지만 저는 끝까지 전혀 안 느끼했고요 너무 맛있어서 말 그대로 흡입 / 순삭하고 나왔어요😂 여러분들 꼭 가세요 진짜 맛있어요 강추! 그리고 저는 대식가라서 밥도 리필해서 먹었는데 심지어 리필 밥도 맛있어🫠 정신 안 차리면 밥 리필 무한으로 퍼먹을 수도 있었을 것 같아요 (이미 양 보면 세그릇은 먹은 듯) 만천원에 일케 배부를 수 있다니 아직 한국 살만하군..! 하 이제 배 두들기면서 중도 가야하는데 걍 주저 앉아서 에비동이나 한그릇 더 푸파하고 싶네요 아니면 중도 가서 나이스 샤워 텐동이 지리는 이유로 논문이나 쓰고 싶네요 하지만 현실은 수사학 논문ㅋ 화이팅☄️ / 백만년만의 신촌거리 학생시절 넓고 화려했던거리가 이젠 좁고작아보인다 그래도 음식들은 더 고급스러워진듯 / 튀김퀄이 상당합니다. 새우튀김 연근튀김 맛있었어요 양이 많은편은 아니지만 깔끔+정갈하게 나옴 규동 불맛이 강해서 향이 좋지만 살짝 짜서 호불호에 따라 밥 추가해야할둣 키오스크 주문식이라 혼밥하기 좋은 맛집 / 스페셜 모듬 텐동 주문했습니다! 장국도 같이 나왔어요.  새우 2마리, 오징어, 가지, 깻잎, 느타리버섯, 단호박, 감자, 꽈리고추, 김, 붕장어, 꽃 이렇게 올려주신 것 같아요! 양이 어마무시합니다. 밥은 리필 가능했어요.  튀김은 바삭하고 얇아서 맛있었는데, 텐동이라 먹다보면 물리는건 어쩔 수 없는 듯 합니다ㅜㅜ 맛있긴 한데 특별한 느낌은 잘 모르겠고, 그래도 튀김 상태가 좋아서 평균 이상 합니다👍🏻 텐동 좋아하시는 분들이라면 만족하실 듯 합니다!  그리고 사장님들이 친절하셨습니다. 단체손님이라 시간이 조금 걸렸는데, 연락 주면 미리 튀기신다고 하셨어요! / 나이스샤워가 연대맛집이라고 해서 다녀왔는데 텐동이랑 규동

done


In [7]:
df1.head()

,id,title,score_val,category,descript,reviews,keyword1,keyword2,keyword3
0,35849217,쭈꾸미블루스 신촌본점,4.59,주꾸미요리,신촌 맛집 날치알쌈과 매운 쭈꾸미,노포식당요즘 인기있는곳이래서 왔어요 주꾸미 삼겹 꽃게새우사리추가해서.먹었는데요 양념...,음식이 맛있어요,친절해요,인테리어가 멋져요
1,1749849331,금고기비스트로 신촌,,"육류,고기요리",,지나가는데 너무 예뻐서 술집인 줄 알았는데 알고 보니까 신촌 삼겹살 맛집이더라구요 ...,음식이 맛있어요,고기 질이 좋아요,친절해요
2,1198813653,댄싱홍콩 신촌점,,중식당,,협력사들과 회의 끝내고 오찬 같이했네요 간만에 오니 새로운 메뉴도 생기고.. 사람...,음식이 맛있어요,인테리어가 멋져요,특별한 메뉴가 있어요
3,1071728472,정통집 신촌점,4.32,돼지고기구이,,이 리뷰는 업체로부터 무료 이용권을 제공 받은 대가로 작성한 솔직 후기입니다. h...,음식이 맛있어요,양이 많아요,친절해요
4,1491059869,오적회관 신촌점,,한식,,📺'생방송투데이'와 '맛의 승부사'에도 방영될 정도로 검증된 유명한 💢핫플매장이더라...,음식이 맛있어요,양이 많아요,재료가 신선해요


In [8]:
df1.shape

(301, 9)

In [12]:
df1.to_csv("result1.csv", index=False) #신촌 맛집 검색어 데이터프레임 내보내기 

In [14]:
#불러와보기! 
df_test = pd.read_csv("result1.csv")
df_test

,id,title,score_val,category,descript,reviews,keyword1,keyword2,keyword3
0,35849217,쭈꾸미블루스 신촌본점,4.59,주꾸미요리,신촌 맛집 날치알쌈과 매운 쭈꾸미,노포식당요즘 인기있는곳이래서 왔어요 주꾸미 삼겹 꽃게새우사리추가해서.먹었는데요 양념...,음식이 맛있어요,친절해요,인테리어가 멋져요
1,1749849331,금고기비스트로 신촌,NaN,"육류,고기요리",NaN,지나가는데 너무 예뻐서 술집인 줄 알았는데 알고 보니까 신촌 삼겹살 맛집이더라구요 ...,음식이 맛있어요,고기 질이 좋아요,친절해요
2,1198813653,댄싱홍콩 신촌점,NaN,중식당,NaN,협력사들과 회의 끝내고 오찬 같이했네요 간만에 오니 새로운 메뉴도 생기고.. 사람...,음식이 맛있어요,인테리어가 멋져요,특별한 메뉴가 있어요
3,1071728472,정통집 신촌점,4.32,돼지고기구이,NaN,이 리뷰는 업체로부터 무료 이용권을 제공 받은 대가로 작성한 솔직 후기입니다. h...,음식이 맛있어요,양이 많아요,친절해요
4,1491059869,오적회관 신촌점,NaN,한식,NaN,📺'생방송투데이'와 '맛의 승부사'에도 방영될 정도로 검증된 유명한 💢핫플매장이더라...,음식이 맛있어요,양이 많아요,재료가 신선해요
...,...,...,...,...,...,...,...,...,...
296,1090749936,멘지 신촌점,NaN,일본식라면,NaN,"🍜 토리파이탄 베스트 메뉴를 먹어봤는데 진한 닭육수에,, 탄탄한 면발이... 얼마 ...",음식이 맛있어요,인테리어가 멋져요,혼밥하기 좋아요
297,1290724510,애몽,NaN,"카페,디저트",NaN,토마토소르베 너무 맛있어요❤️ 귀여운 거북이 모양이구요😆 진짜 생과일의 맛이 팍팍 ...,디저트가 맛있어요,특별한 메뉴가 있어요,인테리어가 멋져요
298,13519110,웅네서서갈비,4.43,"육류,고기요리",농림축산식품부 제공 안심식당,옛날에 종종 갔던 나의 최애 맛집💛 이사하고 오랜만에 갔는데도 변함없는 맛이라 너무...,음식이 맛있어요,친절해요,매장이 넓어요
299,1063168964,멜로디 케이크,NaN,케이크전문,NaN,디자인맛집이에요… 케이크 색상을 파스텔톤으로 예쁘게 만들어주셔서 중요한 기념일에 방...,디저트가 맛있어요,친절해요,선물하기 좋아요


약 97분 소요

In [16]:
# 함수 실행
search = "이대 맛집" # 여기에 검색어 입력
df2 = naverMapCrawling(search)

df2.to_csv("result2.csv", index=False) # 내보낼 파일 이름 설정
print("done")

5
<selenium.webdriver.remote.webelement.WebElement (session="ae2025fc9ff8793cdcd5ea5eac2e2c00", element="f.6DC8789A0310DD35025C2D77C896869F.d.1D094BFD28911398CACD029A9ECC1750.e.124")>
76
##################NEW PAGE##################
<selenium.webdriver.remote.webelement.WebElement (session="ae2025fc9ff8793cdcd5ea5eac2e2c00", element="f.6DC8789A0310DD35025C2D77C896869F.d.1D094BFD28911398CACD029A9ECC1750.e.125")>
74
##################NEW PAGE##################
<selenium.webdriver.remote.webelement.WebElement (session="ae2025fc9ff8793cdcd5ea5eac2e2c00", element="f.6DC8789A0310DD35025C2D77C896869F.d.1D094BFD28911398CACD029A9ECC1750.e.126")>
70
##################NEW PAGE##################
<selenium.webdriver.remote.webelement.WebElement (session="ae2025fc9ff8793cdcd5ea5eac2e2c00", element="f.6DC8789A0310DD35025C2D77C896869F.d.1D094BFD28911398CACD029A9ECC1750.e.127")>
70
##################NEW PAGE##################
<selenium.webdriver.remote.webelement.WebElement (session="ae2025fc9ff8793cdcd

아민 이화
4.62
양식
가게설명 없음
음식이 맛있어요
재료가 신선해요
특별한 메뉴가 있어요
제법 더운날, 먼거리를 찾아간 보람이 가득한 날! 이곳을 콕! 찍은 저의 선택에 어깨가 으쓱해진 날! 처음 먹어보는 색다른 음식에 기대도 컸는데 그런 기대를 배이상 충족시켜준곳이에요~♡  뱅쇼에 올리브유를 2대 1로 섞어마시는 요거, 특이했구요~ 이쁘고 다양한 플레이팅에  제대로 대접받는 느낌이라 감동이었어용~^()^ 이 느낌 그대로 먹는내내 행복했습니다~ 모든 음식이 다 만족스러워서 가까운곳에 있다면 사랑하는 사람들 한번씩 다 데려가서 맛보여주고싶네용!!! 최고의 한끼!!!  직원분들도 너무 친절하셔서 이또한 재방문 하고픈 이유입니당~♡ / 처음 먹어보는 음식이라 맛있을지 어떨지 걱정되었는데 걱정이 무색할정도로 굉장히 맛있었어요! 뱅쇼랑 올리브유를 같이 섞은 식전주도 맛있고 새롭더라구요ㅎㅎ서빙해주시고 어떻게 먹는지 방법을 친절하게 설명해주셔서 그 점도 좋았습니다💓 / 결혼 후 15년만에 신촌 나들이 리뷰 보고 선택한 아민 이화 평상시 접하지 못한 음식이라  아이들의 호응을 받지는 못했지만 타국 음식 느끼고 싶을때 특별한 음식 먹고 싶을때 추천드려요 사장님께서 알려주시는대로 드셔 보세요 음식을 배로 즐길 수 있어요 / 작년에 방문후 재방문. 메뉴와 음식나오는 방식이 살짝 바뀌고 가격은 올라간듯. 여전히 건겅한 음식 가격이 아깝지 않은 풍성한 재료 담백한 맛까지~ 이번에도 대만족! 식전주 양이 많아졌고 올리브오일 따로주고 무엇보다 아보카도 풍성한 메뉴에 감탄~ / ☆☆내돈내산☆☆  웨이팅하기 편하게 가게앞 6인용 벤치있고 연락처남기면 순서대로 전화주심  순서되어 입장하면서 테이블 먼저 잡고  가게 입구에서 키오스크로 메뉴 선택하면됨  태어나서 처음 접해본 메뉴 걱정되었는데 정.말 건강하게 맛있는 맛 적당히 매콤하고 느끼하지않음.  낯선메뉴라 어떻게 먹어야할지 모르겠어서 당황했는데 직원분이 테이블마다 서빙해주시면서 설명 친절하게 해주심 빵도 3조각정도라 배고프지않을까ㅡ했지

하늘초밥
4.64
초밥,롤
가게설명 없음
음식이 맛있어요
재료가 신선해요
친절해요
맛있어요. 재료 신선하고 깔끔하고 구성도 좋고 매우 맛있습니다. 친절하고 굿굿 메뉴판에 주류는 다 안적혀 있는데 냉장고에 샴페인이며 등등 여러 술 많아요. 판매하고 있다고 하고 콜키지는 병당 15,000원이라고합니다.  담에 또오면 샴페인에 먹고싶어요. / 하늘초밥 정말 작았을 때부터 갔는데 2년동안 성장하고 2호점까지 내셨네요 ㅎㅎ  2년 전 정도에는 저는 초밥은 정말 맛있었지만 불만족한 부분도 많았는데요 이제는 그런 부분보다 사장님이 친절하시고 발전하고 손님 편의에 최대한 맞춰주시려는 부분이 감사하고 눈에 들어오네요  이사가기 전에 정말 맛있고 즐겁게 먹고 갔습니다  점점 발전하고 익숙해지시는 것 같아요  다시 이 근처로 이사 올 때는 빌딩 사서 건물 통채로 하늘초밥집이기를 기원하며 ~~  +) 술이랑 밥을 잘 안 곁들여먹는데 하늘초밥의 잔술 온사케는 정말 강추합니다 ~~ 위장이 안 좋은데 온사케는 그냥 차처럼 마시면 되어서 너무 너무 좋아요 !! 초밥이랑 먹으면 조합 와따에욤  사장님 번창하세요 화이팅 !! / 제일 좋아하는 초밥집이에요.처음부터 성공하실줄 알았어요. 줄서서 한번 포장한 이후 여럿이줄서서 가면 못먹길래 혼자서도 달려간 맛집이죠ᆢ못먹는것 적는 메모지도 넘 좋구요. #하늘초밥 현장이 역시 맛나네~ #배려 맛나요 👍👍👍👍👍👍👍👍👍👍👍👍👍👍 2호점 축하드리구요ᆢ맛나게 흡입하다 사레걸려서ᆢ밖으로 잠시 탈출 ㅋㅋ 들어오니 연기 모락모락 나는 진한녹차ᆢ한잔도 주셨네요. ' ◡ ' 세심함에 감사합니다. 하늘초밥 대박날줄 알았어요ᆢ👍 / 토요일 오후 5시 25분에 웨이팅 하기 시작해서 오후 6시 정각에 들어갔어요(35분 웨이팅). 별관이 생겨서인지 생각보다 웨이팅을 길게 하지 않았어요. 제 차례가 되면 전화를 해 주시기때문에 앞에서 무작정 기다리지 않아도 돼서 좋았어요.  입장 순서가 돼서 본관에 들어가면 주문을 먼저 하고, 빈 자리를 안내해 주십니다. 저희는 별관으로 안내

가게 분위기도 인도에 온 것처럼 현지 분위기가 많이 나고 카레도 참 맛있었어요!! 저희는 커리 두 개랑 볶음밥 하나랑 난 하나 시켰는데 둘이서 엄청 배부르게 먹고 먹었습니다 ㅎㅎ 양고기 카레는 양고기 향이 엄청 좋아서 맛있었어요 ㅎㅎ갈릭 허니 난도 달달하고 쫄깃쫄깃 마늘향도 많이 나서 맛있었습니다 ^^카레랑도 잘 어울렸어요 근처 사는 직장인이나 학생분들도 많이 오시는 것 같았어요🥰 저희도 다음에 또 올게요 번창하세요~~ / 완전 인도여행 그 잡채~ 친절하고 재료가 하나하나 신선하고 양념도 너무 맛있고 치팅데이에 추천! 정성어리고 깔끔한 담음새가 좋았어요! 자주 방문하겠습니다~ 강추합니다! 가성비도 좋아여^^ / 단골이였는데 학교 졸업하고 못오다가 몇년만에 다시 왔어요. 위치가 더 좋은 곳으로 바뀌었네요! 맛은 여전히 좋아요 잘 먹었습니다🥰 / 처음 먹어본 인도카레~ 생각했던것 보다 훨씬 더 맛있더라구요~ 향신료맛도 전혀 안느껴지구요 닭고기도 충분히 많이 들어있어요~ 저흰 강황밥보단 갈릭난이랑 먹는게 더 좋았어요~ 세트메뉴엔 플레인난이 무료리필이라 굿👍 난은 뜨거워서 찢을수 없을 정도로 갓 구워져 나왔구요 카레도 다 먹을때까진 뜨끈해서 좋았답니다~ 서빙하시는 분도 사장님도 넘 인상좋고 친절하셨구요~ 다음엔 매운카레 먹으러 꼭 가고 싶네요~~~ / 정통 인도요리전문점 이대맛집 아건 너무 좋네요 사장님 친절하시구요 음식 다 맛있어요 난도 여러 종류 다 맛나구요 커리도 호불호 갈리기 않고 맛있어요 샐러드랑 라씨도 맛나구요 사장님이 디테일을 굉장히 신경쓰셔서 음식이 다 맘있는것같아요~^^ 인도 요리 좋아하시는분들 꼭 방문해보세요 ^^ / 인도 음식 처음 먹어보는데 맛있어요!!!!!  런치세트 주문했는데 완전 배불러요!! 커리도 달달하니 맛있고, 고기도 잘 익고 부드럽고 향도 괜찮고, 샐러드도 볼이 깊어서 야채 양이 많았습니다!! 재방문할 것 같아요^^ / 이대 인도커리 맛집이에요. 아건코스 2인 주문했는데 샐러드, 사모사, 탄두리치킨, 치킨 말라이 티카, 커리, 강황밥

연희동 라이라이 본점
별점 없음
중식당
농림축산식품부 제공 안심식당
음식이 맛있어요
특별한 메뉴가 있어요
재료가 신선해요
★★★★☆  칠리가지는 소스도 소스인데, 끝까지 바삭한 튀김이 예술이에요. 가지소가 풍성하게 들어있는건 아닌데, 소스에 간 돼지고기가 넉넉히 뿌려져있어요. 짜장이 별로라는 평이 많아서 짬뽕으로 시켰는데 그냥 쏘쏘~입니다. 별 하나 뺀 건 주차.. 마지막 사진처럼 주차장은 없다고 보시는게 맞아요. 좁은 도로에 저렇게 대라고 하셔서 사고날까, 딱지 끊길까 불안했어요..ㅜㅜ / 칠리가지가 맛있다고 해서 궁금해 와봤어요^^ 역시나 기대에 부응하는 맛이었습니다. 조금 뜨거울 수 있으니 조심히 드세요 가지튀김 안에 만두소처럼 고기가 같이 들어있어요. 정말 맛납니다. 소스에는 마늘이 많이 들어가서 매우 맛있어요. 짬뽕과 볶음밥도 맛있습니다^^ / 칠리가지+볶음밥 도 좋은데 칠리가지+유니짜장+공기밥 최고!! / 맛있습니다  기대한 대로의 맛입니다  더하지도 않고 덜하지도 않은  요즘 기본이 안된집이 많다보니 돈값한다고 생각만 들어도 만족스럽습니다 / 칠리가지와 탕수육이 맛있었어요. 면류도 맛있었습니다. 다만 주방에서 너무 큰소리로 싸우셔서 먹는 내내 불편했습니다. ㅠㅠ 서빙하시는 분도 해줄건 다 해주시는데 싸워서 그러신건지 대답도 없고 뭔가 쎄~한게  서비스적인 면으로는 개선하셔야 할것 같습니다. / 칠리가지! 정말 맛있습니다 제가 먹은 가지 요리중 탑이네요 같이 간 일행도 최고라고 합니다  다만 가격대가 좀있고 간짜장 보다는 짬뽕을 추천 드립니다. 마찬가지로 탕수육보단 가지튀김이고요  저녁에 가시면 생각보다 대기 시간이 있어요 다들 술도 한잔 하느라 가지 하나만 보고 갈만합니다. / 일주일만에 또 방문했어요 😃 저번에 볶음밥에 같이 나왔던 짬뽕국물이 너무 맛있어서 이번엔 짬뽕을 시켜봤는데요,, 면발은 굵기가 좀 있는 편인데도 불구하고 간이 아주 잘 배어있었고, 국물은 역시 시원칼칼 너무 맛있었어요 😘😘😘 / 저번에 여기 왔던 친구가 칠리 새우가 맛있다고

...

건강한찐빵
4.32
찐빵
가게설명 없음
음식이 맛있어요
특별한 메뉴가 있어요
가성비가 좋아요
표고만두와 마라면 포장해왔습니다. 매장식사를 못하는게 아쉬워요 표고만두는 표고와 고기가 어우러져서  맛있었구요👍 마라면은 산초향이 강해서 호불호가 갈릴 것 같습니다 / 만두가 진짜 맛나요! / 손끝의 정성이 느껴지는 표고만두의 우아한 만듦새 / 매장식사가 안되는게 조금아쉽지만 맛있는 표고만두 /  / 하얼빈 출신 사장님이 하시는 생활의 달인 맛집으로 10개에 6000원씩인 가성비 좋은 아현시장 내 중국식 요리 만두집이네요~  만두소가 보이는 만두피이고 속은 알차지만 약간 심심한 맛이네요. 김치, 고기 두 종류이고 만두만 포장 가능해요. 다른 메뉴는 매장 안에서 먹어야 한대요~ / 건강한찐빵 이름은 찐빵이지만 만두 최고 / 🥟 / 여기 표고만두 제 인생만두입니다 진짜로. 육즙 팡팡 넘치고 표고 짱 많이 들어있고 표고 향 많이 나고 그냥 완전 찰떡궁합이예요. 제발 한 번만이라도 드셔보세요 제발제발제발  아 하고 여기는 포장만 됩니다!!! 매장 안에 식탁이 있지만 요즘은 만두만 가능하고 포장만 하신다고 하셨어요. 참고하세요!! / 표고버섯만두맛있어요  다좋은데 춥다고했는데 밖에서 기다리라고하셔서 ㅜㅜ  매우추운날인데 덜덜떨었네요
돈카춘 이대점
4.67
돈가스
가게설명 없음
음식이 맛있어요
양이 많아요
가성비가 좋아요
저녁하기 귀찮아서 포장주문했는데 배달,포장주문이 많으신지 좀 기다렸네요.센스 넘치게 음료 서비스 주셨어요ㅋ / 좋아요 좋아요 좋아요 좋아요 /  / 맛있어요 / 이번에는 맛있었는데 가끔 고기가 이상할 때도 있어요 🥹 / 맛있어요 / 또 먹고싶당~ / 맛있어요 / 청양마요 돈가스가 최애입니당 … ❤️ / 맛있어요
본죽 아현점
4.47
죽
가게설명 없음
음식이 맛있어요
친절해요
재료가 신선해요
친절하다는 리뷰보고 방문했어요 정말 친절하시고 기분좋게 먹고나왔습니다 몸이 아파 어젠 다른지점 방문했을때 2천원쿠폰 얘기없었는데 결제할때 번거로워도 행사 쿠폰 말씀해주시고 어

씨유그릭
별점 없음
카페,디저트
가게설명 없음
디저트가 맛있어요
가성비가 좋아요
친절해요
저희집 최애 그릭요거트에요! 거의 일주일에 한 번은 포장해서 먹고 있어요~ 오늘도 포장하러 갔다가 휘낭시에 영수증 이벤트 참여했어요. 먹기 전 10분 정도 냉동실에 넣어놨고. 딸과 나눠먹으려고 자르는데 칼에서부터 빠작함이 느껴지네요ㅎㅎ 저는 빠작한 식감을 좋아하는데 완전 취향저격이었어요. 딸도 먹어보더니 지금까지 먹어본 휘낭시에 중에 최고래요! 한 입 먹더니 더 먹고 싶다고 아쉬워해요ㅋ 그릭요거트 맛있는거야 당연하고 휘낭시에도 정식으로 출시되면 사러가야겠어요. 정말 맛있네요^^ / 이대역 그릭 요거트 중에 제일 맛있어요! 적당히 꾸덕하고 토핑이 신선해요ㅎㅎ초코칩 스타일도 여기가 제일 맘에 드네요! 또*n간집입니다 ㅎㅎ 그릭 요거트 추천해요! 예전에는 오후에만 열었는데 이제는 오전에도 여시네요! / 그릭 요거트 처돌이인데 다앙한 종류의 그릭 요거트를 먹어봤는데 씨유그릭만큼 맛있는 곳은 없습니당 👍 요즘은 토핑은 집에서 준비하고 대용량으로 사서 먹는데 맛도 가격도 너무 맘에 듭니다 🙂 학교 앞에 씨유그릭이 있어서 너무 좋아요!!!!!!!! / 리뷰는 처음이지만, 이곳을 알게된 작년부터 엄청 자주먹어요!! 가성비도 좋고, 과일도 맛있고 잠깐 문 닫았을 때 못먹어서 아쉬웠는데 요번에 서비스도 챙겨주시고🤍 요기 넘 좋아용😍😍😍 / 다른 그릭요거트 집이랑 다르게 촉촉 그릭요거트가 있어요!! 당연히 꾸덕한 것도 있구요! 가성비 있어서 자주 애용하는 그릭요거트 집입니다 ㅎㅎ / * 그릭요거트 Basic(100g + 토핑 3개) 4,800원  그래놀라 + 청포도 + 바나나 조합 좋아함 주변에 그릭데이, 고스트요거트도 있는데 개인적으론 여기가 젤 맛있고 가성비도 괜찮다고 생각함  예전에는 그릭요거트 만들어서도 먹었었는데 귀찮기 시작한 이후로는 여기 진짜 자주 먹음 새내기 때 많이 먹다가 한참 안갔는데 얼마전에 리뉴얼 하고 나서 다시 가니 더 맛있어진거 같음 신 맛이 덜 남 아무튼 어어어엄청

본죽 이대점
4.42
죽
가게설명 없음
음식이 맛있어요
친절해요
혼밥하기 좋아요
여름에 음식 진짜 조심해야해요.  탈이 심하게 나서 본죽먹고 회복했습니다. 하루온종일 공복에 이온음료만 먹다가 두근두근 한 첫 곡기였어요 다행히 무사하게 지나갔습니다.  깔꼼하게 포장해주시고, 2개 혹은 3개로 소포장 가능해서 너무 좋은것 같아요. 한번에 먹기 부담스러울때 딱입니다! / 속이 부대낄땐 무조건 본죽 야채죽 넘어주는 센스 :) / 맛있어요 자주 이용합니다 / 역시 죽은 본죽 : ) / 가장 맛있는 죽이에요 홍게죽 맛있어요 / 양도 많구 밑반찬도 많이 주시는데.. 죽에서 조개껍질 같은게 씹혀서 뭐지 했는데 계란껍질이 ㅠㅠ / 맛있어요~ / 아플 때 여기가 유일만 희망 / 사장님이 너무 친절하시고 음식 양도 많아서 너무 좋아요!! / 죽이 부드럽고 맛있어요.
라밥 아현점
별점 없음
종합분식
가게설명 없음
음식이 맛있어요
혼밥하기 좋아요
가성비가 좋아요
큰일났다 감자튀김 유혈사태  너무 맛있어 버림..  감자튀김이라 맛있지않을까? 생각해서 도전하게 된 감튀떡볶이.. 꼭 드셔보세요ㅜㅜ.. 계란까지 으깨가지고  한 그릇 맛있게 뚝딱했어용  맵찔이 도 먹을 수 있는 맛👍🏻 / ○등심돈까스 바삭하고 잘 튀겨짐 ○야간 직원분들 친절함 ○혼밥하기 좋음 ○반찬 셀프시스템 좋음(마카로니 맛있어요)  ※테이블이 좀 끈적하고, 양배추 양이 좀 더 많았으면 좋겠음 / 동네에 혼밥하거나  간단하게 식사할만한 곳은  라밥이 최고인듯  다른 식당은 찌개나 어떤 음식을  전문적으로 팔지만 분식이나  종합적으로 있는 곳은 라밥을 찾게 된다.  감자+떡볶이 조합은 옳다. / 야간 직원분들 친절합니다. 반찬 셀프시스템도 좋아요 / 반찬도 셀프로 먹을 수 있고 음식 맛도 좋아요. 혼밥하기 좋습니다. 돈까스 양배추 샐러드 좀 더 넉넉하게 나오면 좋을거 같아요. / 라밥에서 비빔밥은 처음 먹어봐요! 야채 골고루 들어있고, 비빔장을 미리 안 뿌려주셔서 좋아요. 뿌리고 싶은 만큼 self로 뿌릴 수 있읍니다! 😋 

함평면옥
4.19
만두
가게설명 없음
음식이 맛있어요
재료가 신선해요
친절해요
구워먹는 메뉴나 냉면은 어렵다고 하여 불고기전골 시켰습니다. 아이와 2인분 시켜야 한다고 하셔서 그렇게 시켜서 남겼는데, 느끼해서 많이 남겼고, 메뉴판 기준 된장국이 아니라 갈비탕국물 주셨고, 포장용기 여쭤보니 봉지 주셨어요. 직원분들 관리도 하셔야 겠습니다. / 굿 / 가격실화??;;; 갈비탕 25000원;;? 와우 충격적인 가격; 심지어 2층 구석 식당인데;;; 맛은 SoSo 다시는 방문 안할거 같아요 ㅠㅠ / 직원분들 진짜 친절하시구 식사메뉴를 먹었는데 퀄리티가 넘 좋았고요~ 친구는 육개장 먹었는데 밥 두그릇 비웠어요! 제가 간 시간이 저녁 타임 전 5시 정도라 친구랑 둘이서 오붓하게 식사 잘 하고 왔어요:) / 국물이 자극적이지 않고 짜지않아요~ 14000원 평양식 온반안에 큰인삼뿌리 1개, 대추랑 왕만두랑 밥이랑 다 나옵니다. 비냉도 싹싹 먹었네요~ 식초랑 겨자를 넣지 않아도 간이 딱 좋아요! / 만두 전골 맛있고 , 반찬이 다 맛있어요!! / 굿 / 갈비탕이 너무 맛있었고 고기가 너무 부드러워서 살살 녹는다는 말이 무엇인지 이해할 수 있을 정도였어요! / 좋아요 / 먹을만해요요옹 냉면이 좀 늦게 나오는거 빼고용
봉추찜닭 이대점
4.32
찜닭
농림축산식품부 제공 안심식당
음식이 맛있어요
친절해요
매장이 넓어요
너무 맛있어요!!! / 신촌이대쪽 가면 무조건 가는 단골집이예요 요즘 찜닭처럼 달지않고 마른고추의 칼칼함이 좋아요 감자 당근 닭고기 잘먹고 누룽지 2개를 다먹었어요 사장님 엄청 친절하시고 동치미리필까지 잘먹고 갑니다 / 굿 / 존맛 / 맛있어요 / ● 찜닭 가게 중 가장 자주 들르는 곳이에요. 맛이 늘 한결같아요.  ● 누룽지나 치즈 중 하나를 꼭 추가해서 드시기를 추천합니다. 둘 다 시키면 양이 너무 많고요.  ● 일하는 아주머니는 친절하신데, 카드 분할 결제하는 방법 정도는 배워 두셨음 좋겠어요. / 좋아요 / 원래 맛있어서 종종 찾던 맛집인데... 이날은 유난히 

이디야 신촌기차역점
4.44
카페
가게설명 없음
커피가 맛있어요
친절해요
음료가 맛있어요
가까운 한강 피크닉 전에 들러서 포장주문했어요 매장이 엄청 깨끗하고 신속하게 준비되어 있었어요 옥수수빙수 대존맛 1인1빙수 필수 / 더워요 6월인데 벌써 이리 덥네요 매일 아아 한잔은 마시네요 ㅎㅎ /  / 사장님이 친절하십니다. / 수박주스는 이디야~ 시럽 향 안 나고 맛있어요 / 그래.. 역시 카페는 이디야지... / 언제가도 친절하셔요! / 조아요🌽🍨 / 좋아요 / 가성비 좋아요
done


In [18]:
# 함수 실행
search = "신촌 이대 맛집" # 여기에 검색어 입력
df3 = naverMapCrawling(search)

df3.to_csv("result3.csv", index=False) # 내보낼 파일 이름 설정
print("done")

4
<selenium.webdriver.remote.webelement.WebElement (session="0410a4538e3babb5a5be8b3ae025ceff", element="f.AA9EEC481B1D01B6A966982A012FD386.d.002E66C92407E5EF468A27E7D1062AFE.e.120")>
69
##################NEW PAGE##################
<selenium.webdriver.remote.webelement.WebElement (session="0410a4538e3babb5a5be8b3ae025ceff", element="f.AA9EEC481B1D01B6A966982A012FD386.d.002E66C92407E5EF468A27E7D1062AFE.e.121")>
70
##################NEW PAGE##################
<selenium.webdriver.remote.webelement.WebElement (session="0410a4538e3babb5a5be8b3ae025ceff", element="f.AA9EEC481B1D01B6A966982A012FD386.d.002E66C92407E5EF468A27E7D1062AFE.e.122")>
63
##################NEW PAGE##################
<selenium.webdriver.remote.webelement.WebElement (session="0410a4538e3babb5a5be8b3ae025ceff", element="f.AA9EEC481B1D01B6A966982A012FD386.d.002E66C92407E5EF468A27E7D1062AFE.e.123")>
45
##################NEW PAGE##################
호루몬우직 신촌 본점
별점 없음
일식당
가게설명 없음
음식이 맛있어요
친절해요
특별한 메뉴가 있어요
신촌야끼니꾸 맛집이 거의 없는데 호루몬우

청년밥상문간 이화여자대학교점
별점 없음
한식
가게설명 없음
음식이 맛있어요
가성비가 좋아요
혼밥하기 좋아요
이문수 가브리엘 신부님 존경합니다. 유퀴즈 에서 뵙고 꼭 가보고 싶었는데 드디어 이대점 방문 김치찌개가 3,000원 라면사리 1,000원 스팸사리 1,000원 5,000원에 공기밥도 무료에 콩나물 무침까지 맛을 떠나서 너무나 행복한 한끼식사였습니다 잘먹었습니다 / 정말 맛있고 가성비도 최고입니다! 솔직히 가성비라는 말 쓰기도 죄송할 정도에요..! 가난한 대학생들의 지갑을 지켜주셔서 항상 감사합니당 / 몇년 전에 신부님 책 읽고 정동 꼭 가보고 싶었는데 학교 앞에도 있어서 드디어 방문해보았네요ㅎㅎ 김치찌개 맛있고 가격도 저렴해서 자주 방문하러 갈 것 같아요! 안내해주시는 분들도 친절하셔서 추가 주문한 계란후라이도 맛있게 해먹었습니다🍳 / 이화여대 앞에서 혼밥하기 좋은 공간입니다~ 김치찌개 3천원인데 라면사리와 스팸 추가하면 5천원에 부대찌개를 즐길 수 있습니다! 밥과 반찬도 무한리필이라 정말 든든하게 한끼 먹을 수 있었습니다 :) 혼밥하기 너무 좋은 공간입니다! / 몇년전, 티비에서 청년들을 위해 신부님이 김치찌개를 3천원에 판매 한다는걸 본적이 있었습니다. 우연찮게 신촌 골목길을 거닐다가 발견해서 너무 신기해서 바로 입장해서 먹었어요 국내산 재료라 너무 맛있어서 행복하게 먹고 나왔습니다!!  우리 청년들을 위해 봉사하시는 모든분들 존경 합니다. 고맙습니다❤️ / 김치찌개에 스팸 추가해서 먹었어요! 이게 4천원이라는게 말이 안됩니다🥺🥺 맛있어요!! 고기, 두부도 꽤 들어있구 졸이면서 먹으니까 더 맛있었어요! 혼자 먹기에 양도 많았습니다! 그리고 밥이랑 콩나물 자유롭게 가져다 먹을수있어서 좋아요 콩나물이 짭조름하고 아주 맛있어용 / 가성비가 정말 좋아요. 지갑 사정 좋지 않을 때 가면 너무 좋습니다.. 존재 자체가 감사한 식당이랄까요..ㅎㅎ 받아서 한 번 더 푹 끓여 먹으면 정말 맛있어요!! 두부사리 추가 추천드립니장 / 오늘 방문 했는데 맹물에다가 신맛 첨

솔리드웍스 아이스크림 이대본점
별점 없음
카페,디저트
가게설명 없음
디저트가 맛있어요
특별한 메뉴가 있어요
친절해요
리조는 솔리드웍스가 제일 맛있는 것 같아요✨ 쫀득하고 맛있어서 자주 가는 젤라또 맛집이에요:) 종류도 계절과일도 있고 다양해서 고를 때마다 행복한 고민을 하게 돼요! / 젤ㄹㅏ또_ 라 쓰고 솔리드웍스 아이스크림 & 🥇 아이스걸크림보이_ ㄹㅏ🥈 읽_는_ㄷ ㅏ🥉  이 두ㄱㅏ게ㄱㅏ [체인 젤라또]_중 탑1,2티ㅇㅓ급  ㄱㅏ게인 부분 같_은 👍🏆  🎖토마토 맛은 말해뭐해고 (자주먹는맛)  블루바나나...맛__존맛탱 👍 JMT 🏅 / 너어무 맛있어요!! 숟가락 꽂아주실때 다른맛을 꽂아주시는 직원분의 센스^^ 정말 부드럽고 맛있고 사진도 잘 나오고~ 다음에 또 방문할게요^^ / 이틀 연속 솔리드웍스 제일 좋아하는 젤라또집 / 쌀이 제일 맛있다고 해서 쌀이랑 얼그레이 도전! 맛있어용🍦나중에는 토마토도 먹어봐야겠어요🍅 / 이제는 단골이라고 해도 될 듯.. / 모히또맛이랑 구운 피스타치오맛 먹었는데 엄청 맛있네요.. 사장님 픽으로 랜덤 맛보기도 주십니다 왕맛 굿맛 퀸맛 / 너무 맛있고 서비스도 정말 감사했습니다! 헤이즐넛 넣어주신게 정말 맛있어서 다음에는 사먹어보려구요! / 져는 초코랑 피스타치오가 넘 맛있었어요 / 가게 내부도 예쁘고 젤라또도 맛나용
그릭데이 이대본점
4.72
카페,디저트
가게설명 없음
디저트가 맛있어요
특별한 메뉴가 있어요
매장이 청결해요
민트초코 맛 그릭요거트 어떤 맛일까 궁금해서 주문해봤습니다!  베라 민초 좋아하면 누구나 좋아할 맛입니다 요거트의 신맛없이 민초와 너무 잘 어울립니다  차갑지 않은 민초 아이스크림이라고 할까요,,ㅎㅎ  잘 먹었습니다 또 올게요💓 / 이화여대에 15학번으로 입학하고, 그릭데이가 생기는 순간부터 애정했어요! 그릭데이 덕분에 그릭요거트에 눈을 떴는데, 그릭데이가 저의 입맛과 기준을 너무 높게 형성해줬어요. 다른 곳에서 비슷한 가격으로는 진짜 만족하기가 쉽지 않더라구요 ㅠㅠ 그래서 이대나 홍대, 잠실 갈 때마다 배

도네누볏짚등삼겹살이대점
별점 없음
돼지고기구이
가게설명 없음
키워드 없음
리뷰가 충분하지 않음

꾸덕타코 이대점
별점 없음
일식당
가게설명 없음
음식이 맛있어요
가성비가 좋아요
친절해요
이대쪽은 늦게까지 영업하는 곳이 없어서 야근하면 먹을게 제한적인데 꾸덕타코는 밤에도 영업해서 좋았어요🌃 타코야끼도 맛있고 14알 주문하고 리뷰이벤트로 1알 추가로 받으면 한끼식사로 배부르게 먹었어요😋 다음에 또 올게요! / 꾸덕타코! 이대 뒷골목 귀여운 타코야끼집이에요ㅎㅎ 종류도 많구 따끈한 타코야끼에 맥주한잔 야식으로 제격!! 대파 토핑 추가했더니 풍부한 맛을 즐길 수 있었어요 ㅎㅎ마감시간에 갔더니 하나 더 주셔서 맛있게 잘 먹었습니다!! 문어도 하나가 아니라 두세개는 씹히고 진짜 최고에요!! / 진심 이대 주변 타코야끼 탑티어… 바삭한 거보다 오사카st…까진 아니지만 촉촉 말랑 타코야끼 좋아하시는 분들은 무조건 좋어할 식감입니다 작년보다 문어고 더 커진 느낌? 입 안에 가득차서 너무 좋았어요 불닭이랑 먹을 때는 오리지널이 달달해서 딱입니다 / 이쪽 오길 기다렸다가 드디어 오랜만에 먹었는데 여전히 정말 맛있어요 🫶🏻 문어가 진짜 한가득한 사이즈.. 최고예요 오랜만에 오니까 네온사인도 생겼네요 !! / 이틀 연속으로 꾸덕타코 먹어요ㅎㅎ 어제 매콤 먹고 반해서 매콤이랑 갈릭마요 반반 주문했는데 둘다 너무너무 맛나요ㅠㅠ ㄹㅇ 꿀조합임 소스가 진짜 맛있고, 타코야끼 한입에 넣으면 입안가득 문어가 차서 쫄깃해서 넘맛나요💖💖  한번 꽂히면 계속 먹는데 큰일났습니다 ㅋㅋ / 집근처여서 와봤는데 너무 맛있어요! 왜 진작 안왔는지 후회했습니다ㅜ 사장님도 너무 친절하시고 밖에서 보이는 가게 내부도 청결해보여서 좋았어요 단골이 될 것 같습니다…😍 / 허버허버 먹어서 사진은 없지만, 진짜 이근방 최애 타코야끼집이라 n번째 또간집이에요!!! 모든 메뉴 정복하려고 오리지널부터 매콤까지 왔는데, 매콤먹고 빠져버려서 다른 맛을 도전하지 못하고 있어여ㅋㅋ 콘옥수수가 킥이고 타코야끼가 꾸우덕해서 너무 맛나요

페이브커피 이대 김소영점
별점 없음
카페,디저트
가게설명 없음
커피가 맛있어요
가성비가 좋아요
음료가 맛있어요
저 아보카도 너무 좋아해서 TT 쥬시에서 항상 아보카도바나나 먹었는데 다른 카페에서 많이 안 팔아서 슬펐어요. 근데 여기 아보카도 메뉴가 있는데 심지어 커피TT커피도 마시고싶고 아보카도 음료도 먹고 싶을 때 딱이에요..🥹🤍 사장님도 친절하시고 위치도 너무 좋아요!!💗 수박도 음료말고 밥먹고 디저트로 먹을 수 있게 소분해서 파는 게 너무 좋은 것 가타유 !!! 앞으로 단골할 거에요😽🎀 / ❥ menu: 페이브 다방커피 (1,900)  가성비도 좋고 맛있고, 사장님도 친절하셔서 자주 가는 페이브커피 -!   다방커피는 더위사냥에 커피 섞은 맛이었어요 ◡̈ 달달하고 시원해서 맛있었습니다  다만 개인적으로 ‘페이브 아인슈페너(4,000)’이 더 맛있었던 것 같아요 ! / 스페셜티 커피 좋아하는데 이대앞에서 가성비좋게 먹을수있게돼서 넘좋네용. 시그니처 오렌지 아인슈페너 짱! 추천합니다! 소금빵도 쫄깃하니 맛있어용 ㅎㅎ 맛도리 ~~ / ⭐️⭐️⭐️⭐️ 수박주스 맛있음 투썸보다 맛있다고 느낌 / 수박주스 저렴하고 맛있습니다!! / 커피 진짜 너무 좋아요 커피말고 다른 것도 후기 좋아서 궁금한데 시험기간이라 눈물만 난다 / 아보카도 커피 스무디 설탕 없이 꿀 넣음 아보카도 커피 조합 코커스와 같은 매력 오늘은 알바 분이 계시더라고요 여기 사장님 엄청 친절합니다ㅠㅠ🥰 그리고 이거 계속 먹고 싶었는데, ㄹㅇ 존맛탱 근데 재료가 건강해!!(꿀은 눈 감아) 이런 음료, 건강한데 맛있고 특이한 거 너무 좋습니다  근데 알바 분 좀 힘드실 듯. 주문 폭탄을 맞으신 것 같아요. / 이벤트 50%해서 저렴하게 샀어요  바닐라빈은 진합니다~  캔디 에이드는 유자에이드+사탕 느낌 / 가격이 너무 좋고 친절하신데 (아메리카노 기준) 커피가 맛있진 않아요 맛이 중요하신 분들에게는 비추입니다 가성비는 최고에요! / 키오스크로 연한 커피, 헤이즐넛 커피, 시럽 추가 등 다 되고 맛도 좋아서

...

굿바이슈가베어 포케&샐러드
별점 없음
다이어트,샐러드
가게설명 없음
음식이 맛있어요
재료가 신선해요
혼밥하기 좋아요
저번에 브레이크타임이라 못 갔었는데 이번에 시간이 딱 맞아 로지클라우드 먹어봤는데 맛있네요! 가성비 좋고 구성도 좋아요! 다음에 또 오고 싶어요😃😃💕 / 최고의 포케 맛집👍 굿슈베 갈 생각에 기분좋게 집을 나섰습니다💗✨✨ / 양도 많고 만족합니다! / 너무너무 맛있어요 먹는 내내 행복했어요ㅠㅠ 진짜 최고 포케가 이렇게 맛있는 음식이었다니 매일 먹고싶어요💗 자주 갈게요 맛있는 음식을 먹을 수 있게 해주셔서 감사합니다 / 미스티살사~~너무 너무 맛있어요! 매일먹어도 질리지 않을 것 같아요! / 라이티마이티(₩7,900) 건강한 점심 한 끼로 추천합니다! / 시머링왈츠는 신이다 / 맛있어요! / 맛있고 가격도 괜찮아서 종종 올 것 같아요ㅎㅎ 근데 날벌레가 날라다니고 청결해보이지 않아서 그 점이 조금 아쉬워요ㅠㅠ / 음식이 빨리 나와요
경성양꼬치 신촌직영점
별점 없음
양꼬치
가게설명 없음
음식이 맛있어요
고기 질이 좋아요
친절해요
얼마전에 생긴 신촌 양꼬치 집으로 처음 방문해봤습니다. 6개월 미만의 호주산 양이라는데 고기 질이 상당히 좋았습니다. 양 잡내 같은 것도 없고 맛있었습니다. 양꼬치, 마라갈비, 어깨갈비를 주문을 했고. 마라갈비는 맛있어가지고 추가로 더 주문해서 먹었네요. 사이드로 먹은 옥수수국수도 좋았습니다. 쯔란은 모자랄때마다 직원분께서 채워주시고, 다들 친절하셔서 참 좋았네요. / 신촌에 새로 오픈한 경성양꼬치 !😋 매장도 너무 깔끔하고 칸막이가 설치되어있어 프라이빗해서 좋았어요 ! 메뉴는 양꼬치랑 어깨갈비 먹었는데 누린내 잡내도 하나 없이 야들야들하더라구요~~ 특히 어깨갈비는 육즙이 팡 튀어나오는 식감이 좋아서 맥주안주로 제격..! 날이 더워서 시킨 옥수수냉면은 김치말이국수같은 느낌의 시원한 살얼음 동동 육수에 톡톡한 옥수수면이 은근 잘어울리더라구요 ! 서비수로 주신 물만두도 맛있었습니다 ㅎㅁㅎ 엄청나게 친절하고 유머러스하신 사장님들덕에

방그레커피
별점 없음
카페
가게설명 없음
디저트가 맛있어요
커피가 맛있어요
친절해요
동네 한바퀴 산책하다가 남자친구 별이었던 카페 왔어요ㅋㅋ 남자친구가 웅지 당구장 10년전부터 있던 가게라고 하던데 같은건물이네요!  밀크티, 블루크림소다, 황치즈케익 먹었어요 블루크림소다의 크림은 아이스크림이었어요 ㅎ 색깔이 예쁘고 아이스크림 녹으면서 층이 샤샤샤 하고 녹는 모습도 예뻤어요  황치즈케익이 진짜 꾸덕하고 맛있었는데 포크가 신기하고 먹기 편했습니다 바자리에서 이야기 나누기 좋았고 포장30%가 있는 것 같아서 담에 포장도 해봐야겠어요☺️ / 친구랑 조용하게 잠시 쉬어가고 싶어 알아보던중 찾았는데요, 음료도 커피도 디저트도 맛있고 무엇보다 조용하고 아늑한 분위기에 힐링했어요... 너무너무 잘 쉬다갑니다. 카공이나 작업하기도 너무 좋은 곳이라 추천드려요 저희도 잠깐 있으려다 음료 두잔씩하고 디저트도 먹으면서 2시간 넘게 있었네용. 추천드립니다! / 조용해서 수다떨기도 좋고 분위기도 좋아요!  이대디저트 맛집 답게 황치즈갸또 맛있고 음료도 맛있네요! 이대역 근처라 접근성도 좋고 자주 오고싶어요! / 대부분 카공하는 분들이라 조용하고 좋습니다. 콘센트도 좌석마다 있어서 작업하기 좋아요. 분위기 좋고 예쁜 카페라 좋습니다! / 디저트, 커피 전부 맛도리 입니다♥️ 시그니처 케이크 완전 꾸덕하고 치즈도 진해요!! 나중에 쿠키도 먹으로 또 올게요!! 이대 카페로 강추 합니다♥️ / 이름부터 귀여운 이대 방그레커피입니다! 디저트를 직접 만드셔서인지 퀄리티들이 다 너무 좋았어요ㅎㅎ 키위주스도 진짜 키위 100%가 느껴져서 비타민 충전 제대로 했습니다~~카페 내부 화장실도 있어서 이용하기 편해요! / 가성비도 너무 좋규 이대오면 들리는 디저트 맛집💕 바치케랑 쿠키 넘넘 맛있구용!! 스콘은 사랑입니다 / 커피랑 디저트가 맛있는 공간이에요! 개인적인 업무나 공부할때에도 좋은 곳이고 수다를 떨기에도 너무좋은 곳입니다!! 분위기가 좋아서 시간가는줄 몰랐어요! 화장실도 내부에 있어서 편했습니다

밀랑스
4.65
카페,디저트
no버터no밀가루no화학첨가물 건강하게 만든 간식 이대 마카롱
디저트가 맛있어요
친절해요
특별한 메뉴가 있어요
케이크 시트가 쫀득하고 많이 달지 않아요🥰 여러모로 선물하기 좋을듯해요 / 늘 맛있어요🤎 / 깔끔하고 맛있어요🤍 / 마카롱과 쿠키 샀는데 맛있어서 단골될 예정입니다! 맛있게 잘먹었습니다! / 크리스마스 연말 분위기내기 좋아요 맛있어서 이용하고 있어요 선물로 주니 너무 좋아해서 기분 좋았어요 / 마카롱 사러 갔다가 고소한 빵냄새에 홀려서 말차갸또와 컵파운드를 구매했는데 말차갸또 한입 먹고 엄청 꾸덕하고 맛있어서 놀랐어요! 다음에는 호박이랑 초코도 먹어보려구요ㅎㅎ 사진은 없지만 오랜만에 먹은 무화과 산딸기도 여전히 맛있었어요 앞으로 자주 갈게요! / 선물용으로 구매했습니다. 디저트가 귀엽고 맛있어보여서 즐겁게 골랐네요 ㅎㅎ 다음엔 제 꺼 사러 또 방문할게요~ / 집이 좀 멀어서 언제 또 가야지 했던 밀랑스를 공휴일이라 딸래미랑 왔어요~ 빙수무디 너무 맛있고~ 요즘 트렌드라는 비건 조각케이크도 처음 먹어봤는데 생무화과가 듬뿍~ 건강하면서도 달달하고 촉촉하고~ 특별한 맛이예요~^^ 쿠키랑 마카롱 잔뜩 사들고 왔네요~♡ / 엄마가 여기 마카롱 노래를 부르셔서 반차까지 내고 오랜만에 방문했습니다. 항상 그 자리에 계셔서 감사해요 ㅠㅠ / 컵케잌 맛있어요~
스튜디오웝 신촌점
4.62
카페,디저트
가게설명 없음
디저트가 맛있어요
인테리어가 멋져요
음료가 맛있어요
세브란스 예약일이라 모처럼 대중교통으로 왔다가 모교 정문 쪽 단골 식당에서 점심 먹고, 캠퍼스 천천히 거닐다가 후문으로 왔어요. 덥고 밀크티가 딱 마시고 싶었는데, 제가 수학한 학관(인문대) 바로 건너에 이런 공간이 생겼네요. 좋아하는 아쌈밀크티도 주문하고 미니 스콘도 받았는데, 둘다 더도 말고 덜도 말고 맛이 딱 좋아요. 원래 이대 오면 가는 밀크티집이 있고, 대학 때부터 좋아하던 밀크티집이 신촌 쪽에도 있는데, 여기가 가성비가 좋아 테이크 아웃하기도 좋으네요. 남매둥이 예비맘

1987신샤브 마포점
별점 없음
샤브샤브
가게설명 없음
음식이 맛있어요
재료가 신선해요
친절해요
아파트주민들이 아이들과 많이오는 느낌ㅋㅋㅋㅋ아기의자하나있고 식기도 요청하면 주십니다. 아기 주기좋은 돈까스가 있어서 좋았어요. 양이 많아서 만족스러웠어요! 딤섬맛은 쏘쏘.. 샤브소스는 딱좋았어요! / 식사하는 내내 노래가 좋아서 기분 좋게 맛있게 먹었어요 ~ 1인 얼큰 시켰는데 맛있었어요 알바생 분이 일을 열심히 해주시고 깔끔하세요 너무 귀여우세요  음식도 맛잇고 노래도 좋고 깨끗하고 쾌적해요❤️ 💯💯💯💯💯💯 / 혼밥하러왔는데 메뉴가 혼자먹기도 너무 좋게 잘 구성되오있어요 맛도 좋구요 매장도 깔끔해요 / 비올때 칼국수 생각나서 자주와요 맛도 좋고 다른곳은 얼큰이 별로 맛이없는데 여긴 얼큰이 더 맛잇어요 / 재료 신선하고 양도 많고~ 맛도 좋고~ 여기 물만두도 맛있고 돈까스도 빠삭하니 맛있어요.. 마지막 볶음밥 꼭 드신길^^ / 아이들도 잘 먹고 먹고난후 속도 편한 샤브샤브입니다~ 샤브샤브 할때 에피타이저로 시킨 돈가스도 바삭하고 아이들이 좋아해요~ 다먹고 칼국수, 죽까지 완벽합니다^^ / 맛잇어요! 점심으로 또 왔어요 칼국수에 볶음밥까지 알차요  또 올거에요 / 샤브샤브 맛집!!  양이 엄척 많아서 배불러요!! ㅎㅎ 맛도 매우 맛있게 먹었어요💖💖 조만간 또 올게요👍🏻👍🏻👍🏻 재료도 신선합니다🥳 / 가성비 좋아요 양 많음!!! 땅콩소스가 매력있씁니다 / 남자 알바생이 너무 귀여워요  혼자 막 바쁘게 뽈뽈뽈 다녀요  손님들을 자주 살펴주고 굉장히 친절하세요 앞치마도 잘 어울리시는 것 같아요 ㅋㅋ  샤브도 맛있고 알바생도 너무 귀여우셔가지고 혼밥하러 자주 올 것 같아요  일도 열심히하고 배달도 척척 잘 준비해둬요  빈그릇도 척척 잘 치워주시고 부족한 거 없나 자주 봐주셔요 웃진 않으시지만 친절하세요
오아시스
별점 없음
퓨전음식
가게설명 없음
인테리어가 멋져요
음식이 맛있어요
친절해요
너무좋았던시간이였어요~ 너무친절하시고 맛있고 노래들으면서 힐링했습니다 또갈예정이예요~ /

Quid
별점 없음
와인
가게설명 없음
인테리어가 멋져요
친절해요
대화하기 좋아요
친구와 너무 편안하고 즐거운 시간 보냈습니다. 음식도 너무 다 맛있고 와인도 잘어울렸어요 :) 인테리어도 깔끔하고 편안해서 좋았습니다. 이날을 기념할 수 있는 사진인화 서비스까지 완벽한 저녁시간이였어요! 조만간 또 방문할게요😍 / 맛있어요!! 아직 사람도 많이 없어서 좋아요👍🏻 네이버 예약 되는것도 편하고 와인도 맛있네요. / 분위기도 너무 좋았던 퀴드! 대학가 근처라 그런지 와인 가격대도 괜찮고 특히 맛이 너무너무 좋았어요ㅠㅠ 재방문 의사 95726947262개 입니다..🫶🏻 이대 근처에서 분위기 괜찮은 곳 찾으시는 분들 그냥 퀴드 가세요..💨 / 분위기 좋고 음식도 맛있었어요 다음에 또 방문할 예정입니다~ / Wine & work station으로 점심에 방문했어요! 햇빛이 퍼지며 창문으로 들어오는 초록색 빛과 스피커로 나오는 피아노 곡들이 분위기를 완성해주었습니다! 집중해서 책한권 거의 다 읽을 수 있었어요. 함께 주신 화이트와인도 제 입맛엔 맞았는데 호불호가 있을 것 같았어요.  전 단체 좌석에서 있었는데 오래 있기엔 등받이가 없어서 조금 힘들었지만 나름대로 괜찮았습니다! / 이렇게 까지 사진을 많이 올려본 적이 없는데 정말 너무 너무 좋았어요 !! 🤍 테이블 사이 거리도 넓직하고 인테리어 하나하나 , 조명도 다 예쁘고 이곳 저곳 신경쓰신 티가 많이 나요 !!!  와인이랑 안주 메뉴도 너무 합리적이고 좋았어요 !! 화장실도 깔끔하고 그냥 다 완벽해요 !!  찾아오기는 여기가 긴가민가 하면서 들어가야합니다 5층에 있는데 엘레베이터가 없어서 조오큼 힘들었는데 매장 들어가자마자 너무 예쁘고 아늑해서 삭 잊었어요 !!!!! 재방문 의사 20000000% 입니다 ! ✨ / 날 좋은 4월의 토요일, work & wine 너무너무 좋았어요✨ 차분하고 포근한 분위기에서 즐겁게 작업하다 갑니다 :) / 이대 근처 분위기 좋은 와인바입니다. 와인 종류가 다양하고 안주들 가격이 좋았습니다. 

In [1]:
import pandas as pd

In [2]:
test1 = pd.read_csv('result1.csv')
test2 = pd.read_csv('result2.csv')
test3 = pd.read_csv('result3.csv')

In [3]:
print(test1.shape)
print(test2.shape)
print(test3.shape)

(301, 9)
(310, 9)
(247, 9)


In [5]:
#전체 결과 concat 
final_result = pd.concat([test1, test2, test3], ignore_index = True)
print(final_result.shape)
final_result.head()

(858, 9)


,id,title,score_val,category,descript,reviews,keyword1,keyword2,keyword3
0,35849217,쭈꾸미블루스 신촌본점,4.59,주꾸미요리,신촌 맛집 날치알쌈과 매운 쭈꾸미,노포식당요즘 인기있는곳이래서 왔어요 주꾸미 삼겹 꽃게새우사리추가해서.먹었는데요 양념...,음식이 맛있어요,친절해요,인테리어가 멋져요
1,1749849331,금고기비스트로 신촌,NaN,"육류,고기요리",NaN,지나가는데 너무 예뻐서 술집인 줄 알았는데 알고 보니까 신촌 삼겹살 맛집이더라구요 ...,음식이 맛있어요,고기 질이 좋아요,친절해요
2,1198813653,댄싱홍콩 신촌점,NaN,중식당,NaN,협력사들과 회의 끝내고 오찬 같이했네요 간만에 오니 새로운 메뉴도 생기고.. 사람...,음식이 맛있어요,인테리어가 멋져요,특별한 메뉴가 있어요
3,1071728472,정통집 신촌점,4.32,돼지고기구이,NaN,이 리뷰는 업체로부터 무료 이용권을 제공 받은 대가로 작성한 솔직 후기입니다. h...,음식이 맛있어요,양이 많아요,친절해요
4,1491059869,오적회관 신촌점,NaN,한식,NaN,📺'생방송투데이'와 '맛의 승부사'에도 방영될 정도로 검증된 유명한 💢핫플매장이더라...,음식이 맛있어요,양이 많아요,재료가 신선해요


In [6]:
#겹치는 id 개수 확인 
final_result['id'].value_counts()

18232012      3
1608001428    3
36631076      3
1288609263    3
1946223191    3
             ..
1922617213    1
1250906147    1
1984155649    1
1970380658    1
89888882      1
Name: id, Length: 694, dtype: int64

In [14]:
#id 중복 제거 
non_dupl = final_result.drop_duplicates(['id'], ignore_index = True)
non_dupl.head()

,id,title,score_val,category,descript,reviews,keyword1,keyword2,keyword3
0,35849217,쭈꾸미블루스 신촌본점,4.59,주꾸미요리,신촌 맛집 날치알쌈과 매운 쭈꾸미,노포식당요즘 인기있는곳이래서 왔어요 주꾸미 삼겹 꽃게새우사리추가해서.먹었는데요 양념...,음식이 맛있어요,친절해요,인테리어가 멋져요
1,1749849331,금고기비스트로 신촌,NaN,"육류,고기요리",NaN,지나가는데 너무 예뻐서 술집인 줄 알았는데 알고 보니까 신촌 삼겹살 맛집이더라구요 ...,음식이 맛있어요,고기 질이 좋아요,친절해요
2,1198813653,댄싱홍콩 신촌점,NaN,중식당,NaN,협력사들과 회의 끝내고 오찬 같이했네요 간만에 오니 새로운 메뉴도 생기고.. 사람...,음식이 맛있어요,인테리어가 멋져요,특별한 메뉴가 있어요
3,1071728472,정통집 신촌점,4.32,돼지고기구이,NaN,이 리뷰는 업체로부터 무료 이용권을 제공 받은 대가로 작성한 솔직 후기입니다. h...,음식이 맛있어요,양이 많아요,친절해요
4,1491059869,오적회관 신촌점,NaN,한식,NaN,📺'생방송투데이'와 '맛의 승부사'에도 방영될 정도로 검증된 유명한 💢핫플매장이더라...,음식이 맛있어요,양이 많아요,재료가 신선해요


In [15]:
print(non_dupl.shape)
non_dupl['id'].value_counts()

(694, 9)


35849217      1
1164286157    1
21853263      1
13371590      1
1814409723    1
             ..
37663862      1
35915198      1
13154001      1
1088886581    1
89888882      1
Name: id, Length: 694, dtype: int64

In [16]:
non_dupl['title'].value_counts()

쭈꾸미블루스 신촌본점        1
계절의 목소리            1
앨리스                1
헐리우드               1
진담                 1
                  ..
또보겠지떡볶이집 붕붕허니비점    1
거북이의주방             1
술익는마을              1
곱창이야기 신촌점          1
보라네도시락             1
Name: title, Length: 694, dtype: int64

In [17]:
non_dupl.tail()

,id,title,score_val,category,descript,reviews,keyword1,keyword2,keyword3
689,1056075705,1987신샤브 마포점,NaN,샤브샤브,NaN,아파트주민들이 아이들과 많이오는 느낌ㅋㅋㅋㅋ아기의자하나있고 식기도 요청하면 주십니다...,음식이 맛있어요,재료가 신선해요,친절해요
690,38705128,미스터 참치 마포직영점,NaN,생선회,NaN,생일 저녁으로 남자친구랑 동네 가까운 곳에서 뭐먹을지 고민하다가 갑자기 참치가 땡겨...,음식이 맛있어요,재료가 신선해요,가성비가 좋아요
691,1934848485,Quid,NaN,와인,NaN,친구와 너무 편안하고 즐거운 시간 보냈습니다. 음식도 너무 다 맛있고 와인도 잘어울...,인테리어가 멋져요,친절해요,대화하기 좋아요
692,1283043137,즐겁다,NaN,"맥주,호프",NaN,바로 집앞이라 자주 가는 술집이에요. 납작만두 야채 완전 새콤하네요 ㅎㅎ 만두가 1...,음식이 맛있어요,친절해요,특별한 메뉴가 있어요
693,89888882,보라네도시락,NaN,"도시락,컵밥",NaN,NaN,NaN,NaN,NaN


In [18]:
non_dupl.to_csv("final_result.csv", index=False) 

In [21]:
non_dupl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694 entries, 0 to 693
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         694 non-null    int64  
 1   title      694 non-null    object 
 2   score_val  411 non-null    float64
 3   category   694 non-null    object 
 4   descript   197 non-null    object 
 5   reviews    688 non-null    object 
 6   keyword1   670 non-null    object 
 7   keyword2   670 non-null    object 
 8   keyword3   670 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 48.9+ KB
